In [ ]:
#zoveel mogelijk in 't nederlands om makkelijk te zien wat keywords en woorden uit de data zijn
#zoveel mogelijk in functies om code makkelijk te herbruiken/lezen
#settings apart om makkelijk te tweaken

import requests
import json
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt #TODO uitzoeken hoe dubbele import te fixen

#settings apart om te tweaken
aandeel = "SOLB.BR"
begin = 1024383600
einde = 1592031600 #ook mogelijk: einde = begin + tijdsduur_in_seconden
interval = '1d'
marktkost_perc = 0.0035
brokerkost_euro = 7.5
beginbedrag = 1000
omvangprentje = [20,5]

#code wat spreiden in functies
#fetch de data, returnt: (timestamps in epoch-secs, closeprijzen)
def fetchdata(aandeel, begin, einde, interval):
    #fetchen
    session = requests.session()
    url = 'https://query2.finance.yahoo.com/v8/finance/chart/'
    url += aandeel + '?symbol=' + aandeel
    url += '&period1=' + str(begin) + '&period2=' + str(einde)
    url += '&interval=' + interval
    #parsen
    data = session.get(url).json()['chart']
    if data['error'] != None:
        quit() #probleem bij data ophalen
    data = data['result'][0]
    tijden = [datetime.fromtimestamp(tijdstip) for tijdstip in data['timestamp']]
    return (tijden, data['indicators']['quote'][0]['close'])

#Bij Null-waarden in de prijzen de laatst bekende prijs nemen (of de eerst bekende indien Null de 1ste prijs is)
def fixlegeprijzen(prijzen):
    for i, prijs in enumerate(prijzen):
        if prijs == None:
            if i > 0:
                prijzen[i] = prijzen[i-1]
            else: #als eerste een null is verder zoeken tot 1ste echte waarde
                j=1
                while(prijzen[j] == None and j<len(prijzen)):
                    j+=1
                prijzen[0] = prijzen[j]
                if prijzen[0] == None: #Helemaal geen prijzen
                    quit()

#extra kosten bij (aan/ver)koop aan een bepaalde koers
def kosten(koers):
    return koers*marktkost_perc + brokerkost_euro

class Portofolio:
    def __init__(self, beginbedrag):
        self.geld = beginbedrag
        self.aankooptijden = []
        self.verkooptijden = []
        self.aantaandelen = 0
    def koop(self, prijs, time_counter): #koop 1 aandeel
        if self.geld >= prijs + kosten(prijs): #niet in't rood gaan...
            self.geld = self.geld - prijs - kosten(prijs)
            self.aankooptijden.append(matplotlib.dates.date2num(tijden[time_counter]))
            self.aantaandelen += 1
    def verkoop(self, prijs, time_counter): #verkoop 1 aandeel
        if self.aantaandelen > 0: #niet shorten
            self.geld = self.geld + prijs - kosten(prijs)
            self.verkooptijden.append(matplotlib.dates.date2num(tijden[time_counter]))
            self.aantaandelen -= 1

def uitvoer(portofolio):
    #tekst
    print('Eindwaarde:', round(portofolio.geld,2),'euro')
    print('Totaal aantal aandelen:', portofolio.aantaandelen)
    print('Eind kapitaal:', round(totalen[-1],2),'euro')
    print('Winst:', round((totalen[-1]-beginbedrag)/beginbedrag,2),'%')
    #grafieken van ...
    matplotlib.rcParams['figure.figsize'] = omvangprentje
    datums = matplotlib.dates.date2num(tijden)
    fig, ax = plt.subplots(nrows=1,ncols=2)
    # ... koers over tijd + (aan/ver)koop acties
    ax[0].plot_date(datums, sluitprijzen,'-',marker='o')
    ax[0].title.set_text('Koers + koop (rood) en verkoop (groen) tijdstippen')
    for aankooppunt in test.aankooptijden:
        ax[0].axvline(x=aankooppunt,color='r',linestyle='--')
    for verkooppunt in test.verkooptijden:
        ax[0].axvline(x=verkooppunt,color='g',linestyle='--')
    # ... kapitaal over tijd
    ax[1].plot_date(datums, totalen,'-')
    ax[1].title.set_text('Totaal kapitaal (inclusief aandelen)')
    plt.show()

(tijden, sluitprijzen) = fetchdata(aandeel, begin, einde, interval)
fixlegeprijzen(sluitprijzen)
test = Portofolio(beginbedrag)
totalen = []
num_subsequent_events = 8
for i,value in enumerate(sluitprijzen):
    if i > num_subsequent_events-1:
        close_slice = sluitprijzen[i-num_subsequent_events:i+1]
        if all(i < j for i, j in zip(close_slice, close_slice[1:])):
            test.verkoop(value, i)
        elif all(i > j for i, j in zip(close_slice, close_slice[1:])):
            test.koop(value, i)
    if test.aantaandelen > 0:
        totalen.append(test.geld + test.aantaandelen*(1-marktkost_perc)*sluitprijzen[i] - brokerkost_euro)
    else:
        totalen.append(test.geld)
uitvoer(test)